# Neural Search

An AI powered search engine using Transformers, K-Means, and Cosine Similarity.

In [1]:
!pip install -q transformers

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [2]:
import sagemaker
import pandas as pd

In [3]:
sess = sagemaker.Session()

bucket_name = sess.default_bucket()
role_name = sagemaker.get_execution_role()

bucket_prefix = "neural-search"

## Load data

In [4]:
data_location = f"s3://{bucket_name}/{bucket_prefix}/data/movies_metadata.csv"

df = pd.read_csv(data_location)[["title", "overview"]]
df = df[df["overview"].notna()]

df.head(2)

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...


## Load embedding model

In [5]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

In [27]:
model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)

text_model = BertModel.from_pretrained(model_name)
cosine_similarity_model = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text_model.to(device)
cosine_similarity_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CosineSimilarity()

In [7]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    
    pooled = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    return torch.nn.functional.normalize(pooled, p=2, dim=1)

In [8]:
def create_embeddings(text, tokenizer, model):
    encoded_input = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    return mean_pooling(model_output, encoded_input["attention_mask"])

### Example embedding similarities

In [28]:
from itertools import combinations

sample_text = [
    "Andy is going to the beach on Sunday with his friends",
    "Mandy is going to the movies Tuesday with her Mum",
    "The cargo ship sailed through the night",
]

sample_embeddings = create_embeddings(sample_text, tokenizer, text_model).cpu()

tuples = combinations(list(range(sample_embeddings.shape[0])), 2)

with torch.no_grad():
    for i, j in tuples:
        similarity = cosine_similarity_model(sample_embeddings[i], sample_embeddings[j])
        print(f"'{sample_text[i]}', '{sample_text[j]}', similarity={similarity}")

'Andy is going to the beach on Sunday with his friends', 'Mandy is going to the movies Tuesday with her Mum', similarity=0.8401806950569153
'Andy is going to the beach on Sunday with his friends', 'The cargo ship sailed through the night', similarity=0.561195433139801
'Mandy is going to the movies Tuesday with her Mum', 'The cargo ship sailed through the night', similarity=0.5241331458091736


## Create model pipeline

### Preprocessing
- Take the text from pandas
- Run the model on all of the text
- Record the embedding vectors in the dataframe
- Perform KMeansClustering and record the cluster for each vector

### Perform search
- Create embeddings
- Get cluster of embedding vectors
- Find all text with the same cluster
- Perform cosine similarity search between all elements in the cluster
- Return the top K most elements

In [22]:
df["embeddings"] = np.nan
df["cluster"] = -1

df.head(2)

,title,overview,embeddings,cluster
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",NaN,-1
1,Jumanji,When siblings Judy and Peter discover an encha...,NaN,-1


In [29]:
batch_size = 32

input_text = df["overview"].values

for i in range(0, len(input_text) // batch_size, batch_size):
    batch = list(input_text[i:i + batch_size])
    
    batch_embeddings = create_embeddings(batch, tokenizer, text_model)
    
    print(batch_embeddings)
    
    break

tensor([[-0.0296,  0.0278,  0.0258,  ...,  0.0236,  0.0131, -0.0032],
        [-0.0240,  0.0164,  0.0324,  ...,  0.0049,  0.0090,  0.0282],
        [ 0.0059, -0.0250,  0.0348,  ..., -0.0197,  0.0080, -0.0302],
        ...,
        [-0.0053, -0.0131, -0.0133,  ...,  0.0029,  0.0026, -0.0022],
        [-0.0098, -0.0114,  0.0017,  ...,  0.0073, -0.0028, -0.0029],
        [-0.0252,  0.0081,  0.0026,  ...,  0.0027,  0.0078,  0.0111]])
